#### Auto Encoder

This is the auto encoder implementation that is going to be used in our study, the 

In [21]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import kagglehub # Gather data from kagglehub

In [22]:
# Loading dataset
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
file = os.path.join(path, "creditcard.csv")
df = pd.read_csv(file)
 
print(df.shape)

(284807, 31)


In [23]:
# Data split (organization of training and test data)
x = df.drop(columns=['Class'])  # Extracts all features except target
y = df['Class']  # Extracts only the target feature

# Split dataset into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                    random_state=15, 
                                                    stratify=y)

# Check class distribution in training and testing sets
print("Training set class distribution:\n", y_train.value_counts(normalize=True))
print("Testing set class distribution:\n", y_test.value_counts(normalize=True))

Training set class distribution:
 Class
0    0.998271
1    0.001729
Name: proportion, dtype: float64
Testing set class distribution:
 Class
0    0.99828
1    0.00172
Name: proportion, dtype: float64


In [32]:
# Auto Encoder model
input_dimension = x_train.shape[1]
hidden_layer = 16
input_layer = Input(shape=(input_dimension,))
Encoder1 = Dense(hidden_layer,activation="tanh")(input_layer)
Encoder2 = Dense(int(hidden_layer/2),activation="tanh")(Encoder1)
Encoder3 = Dense(int(hidden_layer/4),activation="tanh")(Encoder2)
Decoder1 = Dense(int(hidden_layer/4),activation="tanh")(Encoder3)
Decoder2 = Dense(int(hidden_layer/2),activation="tanh")(Decoder1)
Decoder3 = Dense(input_dimension,activation="tanh")(Decoder2)
AutoEncoderModel = Model(inputs=input_layer,outputs=Decoder3)

In [33]:
# Compile model
optimizer = Adam(learning_rate=0.001)
AutoEncoderModel.compile(optimizer='adam', loss='mse')

In [36]:
AutoEncoderModel.fit(x_train, x_train, epochs=50, batch_size=512, validation_data=(x_test, x_test))

Epoch 1/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 375059072.0000 - val_loss: 375830240.0000
Epoch 2/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 374015232.0000 - val_loss: 375830240.0000
Epoch 3/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 374519968.0000 - val_loss: 375830240.0000
Epoch 4/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 374385952.0000 - val_loss: 375830240.0000
Epoch 5/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 374856480.0000 - val_loss: 375830240.0000
Epoch 6/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 375204928.0000 - val_loss: 375830240.0000
Epoch 7/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 373920224.0000 - val_loss: 375830240.0000
Epoch 8/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - loss: 374764320.0000 - val_loss: 375830240.0000
Epoch 9/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 374599360.0000 - val_loss: 375830240.0000
Epoch 10/50
446/446 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 374769504.0000 - v